# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [12]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=1)


In [19]:
prompt = ChatPromptTemplate.from_template( "What can {product} do:"
 
)

In [20]:

#chain = LLMChain(llm=llm, prompt=prompt)
chain = prompt | llm

In [71]:
product = "matress"
chain.invoke(product)

{'product': 'matress',
 'text': '1. Provide support and comfort while sleeping\n2. Help alleviate back, neck, and shoulder pain\n3. Improve overall quality of sleep\n4. Help regulate body temperature\n5. Reduce motion transfer between sleep partners\n6. Improve alignment of the spine\n7. Enhance overall health and well-being\n8. Extend the lifespan of your mattress\n9. Customize firmness and materials to suit individual preferences\n10. Contribute to a relaxing and restful sleep environment.'}

## SimpleSequentialChain

In [16]:
from langchain.chains import SimpleSequentialChain

In [43]:
llm = ChatOpenAI(temperature=0.3)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
     "What models of {product} are there?"
 
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [44]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Which of {first_prompt} is the best?"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)


In [45]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [46]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
There are several different models of mattresses available on the market, including:

1. Innerspring mattresses: These mattresses use a system of metal coils to provide support and comfort.

2. Memory foam mattresses: These mattresses are made from viscoelastic foam that conforms to the shape of your body, providing pressure relief and support.

3. Latex mattresses: These mattresses are made from natural or synthetic latex foam, which is known for its durability and responsiveness.

4. Hybrid mattresses: These mattresses combine elements of innerspring, memory foam, and/or latex mattresses to provide a balance of support and comfort.

5. Pillow-top mattresses: These mattresses have an extra layer of padding on top for added softness and comfort.

6. Adjustable mattresses: These mattresses have a motorized base that allows you to adjust the position of the mattress for personalized comfort.

7. Air mattresses: These mattresses use air cham

'The best mattress ultimately depends on individual preferences and needs. Some people may prefer the support and bounce of an innerspring mattress, while others may prefer the pressure relief and contouring of a memory foam mattress. It is important to consider factors such as firmness, materials, and sleeping position when choosing a mattress. It is recommended to try out different mattresses in person to determine which one feels the most comfortable and supportive for you.'

**Repeat the above twice for different products**

## SequentialChain

In [47]:
from langchain.chains import SequentialChain

In [89]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this {review} into german"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translated"
                    )


In [90]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the {translated} focusing on strengths and weaknesses in german"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key= "summarized"
                    )


In [91]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "translate {summarized} to english"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="english_review"
                      )


In [96]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "How good is the quality of the translation from {summarized} and {english_review}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="conclusion"
                     )


In [97]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables= ["review"],
    output_variables=[ "summarized", "english_review", "conclusion"],
    verbose=True
)

In [98]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'summarized': 'Stärken: Der Geschmack der Produkte aus dem Laden ist viel besser als der Geschmack des aktuellen Produkts. \n\nSchwächen: Der Schaum hält nicht, was als seltsam empfunden wird. Es wird vermutet, dass es sich um eine alte Charge oder eine Fälschung handeln könnte.',
 'english_review': 'Strengths: The taste of the products from the store is much better than the taste of the current product.\n\nWeaknesses: The foam does not hold, which is perceived as strange. It is suspected that it could be an old batch or a counterfeit.',
 'conclusion': 'The quality of the translation is good. It accurately conveys the main points of the strengths and weaknesses of the product.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [134]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [135]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

#prompt_templates = {
#    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
#    for info in prompt_infos
#}

In [136]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


In [137]:
llm = ChatOpenAI(temperature=0)

In [138]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [139]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [140]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [130]:

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
    verbose=True
)

router_chain = router_prompt | llm | RunnableLambda(lambda x: x.content)

chain = (
    RunnablePassthrough()
    | {
        "input": RunnablePassthrough(),
        "chosen_prompt": router_chain,
    }
    | RunnableLambda(lambda x: prompt_templates[x["chosen_prompt"]].format(input=x["input"]))
    | llm
)

In [141]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [142]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\Salva\AppData\Local\Temp\ipykernel_61852\3038952769.py:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain,


In [143]:
response = chain.invoke("Explain what principle component analysis is using linear algebra")



> Entering new MultiPromptChain chain...
computer science: {'input': 'Explain what principle component analysis is using linear algebra'}
> Finished chain.


In [144]:
response

{'input': 'Explain what principle component analysis is using linear algebra',
 'text': 'Principal Component Analysis (PCA) is a technique used in data analysis and machine learning to reduce the dimensionality of a dataset while preserving as much variance as possible. \n\nIn linear algebra terms, PCA involves finding the eigenvectors and eigenvalues of the covariance matrix of the dataset. The eigenvectors represent the directions of maximum variance in the data, while the eigenvalues represent the amount of variance along each eigenvector.\n\nTo perform PCA, we first center the data by subtracting the mean of each feature from the data points. Then, we compute the covariance matrix of the centered data. Next, we find the eigenvectors and eigenvalues of the covariance matrix. \n\nThe eigenvectors form a new basis for the data, where the first eigenvector corresponds to the direction of maximum variance, the second eigenvector corresponds to the direction of second maximum variance, a

In [147]:
chain.invoke("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


{'input': 'What is black body radiation?',
 'text': "Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."}

In [145]:
chain.invoke("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


{'input': 'what is 2 + 2', 'text': 'The answer to 2 + 2 is 4.'}

In [146]:
chain.invoke("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**